In [1]:
#libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
import joypy
import requests
import json

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.experimental import enable_halving_search_cv  
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor


import tracemalloc
import time
import yellowbrick
from yellowbrick.regressor import ResidualsPlot, CooksDistance
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.api import qqplot

import lime 
from lime import lime_tabular
import shap
shap.initjs()
import re

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('df_HDB.csv')
y = df[['resale_price']]
X = df.drop(['resale_price','town', 'year'], axis=1)

In [3]:
def tracing_start():
    tracemalloc.stop()
    print("nTracing Status : ", tracemalloc.is_tracing())
    tracemalloc.start()
    print("Tracing Status : ", tracemalloc.is_tracing())
def tracing_mem():
    first_size, first_peak = tracemalloc.get_traced_memory()
    peak = first_peak/(1024*1024)
    print("Peak Size in MB - ", peak)

In [4]:
# Train Test Split
tracing_start()
start = time.time()

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1, shuffle=True, random_state=0)

# Validation using out-of-bag method
rf = RandomForestRegressor(n_estimators=100, oob_score=True, random_state=0)
rf.fit(X_train, y_train.values.ravel())
predicted_train = rf.predict(X_train)


# Display result with train set
print(f'Out-of-bag R\u00b2 training score: {rf.oob_score_}')
print(f'Number of features: {rf.n_features_}')
print(f'Number of output: {rf.n_outputs_}')
end = time.time()
print("time elapsed {} milli seconds".format((end-start)*1000))
tracing_mem()

tracing_start()
start = time.time()

# predict and get evaluation metrics on test set
predicted_test = rf.predict(X_test)
oob_test_score = r2_score(y_test['resale_price'], predicted_test)
oob_mse = mean_squared_error(y_test['resale_price'], predicted_test)
oob_mae = mean_absolute_error(y_test['resale_price'], predicted_test)
oob_rmse = np.sqrt(mean_squared_error(y_test['resale_price'], predicted_test))

# Display result with test set
print(f'Test data R\u00b2 score: {oob_test_score}')
print(f'Test data Mean Squared Error: {round(oob_mse)}')
print(f'Test data Mean Absolute Error: {round(oob_mae)}')
print(f'Test data Root Mean Squared Absolute Error: {round(oob_rmse)}')
end = time.time()
print("time elapsed {} milli seconds".format((end-start)*1000))
tracing_mem()

nTracing Status :  False
Tracing Status :  True
Out-of-bag R² training score: 0.9603461530240771
Number of features: 26
Number of output: 1
time elapsed 76469.86746788025 milli seconds
Peak Size in MB -  63.08407688140869
nTracing Status :  False
Tracing Status :  True
Test data R² score: 0.9617487448508271
Test data Mean Squared Error: 846060541
Test data Mean Absolute Error: 21573
Test data Root Mean Squared Absolute Error: 29087
time elapsed 337.42642402648926 milli seconds
Peak Size in MB -  3.8544998168945312


In [7]:
# Produce a matrix for client data
client_data = [[1, 3, 68, 1986, 0.594128259344289, 15 ,0.22516102677612, 8 ,0.376763931855833,28, 1.5803380574408, 1, 1.1060523511997, 2, 0.274218362591398, 9, 9.26348665949998, 0,0,1,0,0,0,1,0,0], # Client 1
               [2, 2, 93, 2010, 0.173394270589029, 20 ,0.495748156781795, 1 ,0.801641910325716,3, 1.2895891636303, 1, 0.354436984819771, 23, 0.0441655994955788, 17, 13.6128567065317, 0,0,1,0,1,0,0,0,0], # Client 2
               [2, 0, 112, 1995, 0.18934288586454, 17 ,2.03316923897357, 0 ,0.466650001317567,11, 0.398785707885446, 6, 0.97903269235057, 6, 0.413983861488505, 16, 13.782717157755, 1,0,0,0,0,0,1,0,0]]  # Client 3
print('Resale price: $255,000.00, $440,000.00, $385,000.00')
print('Adj. inflation: $268,429.53, $463,172.52, $403,720.52')
# Show predictions
for i, price in enumerate(rf.predict(client_data)):
    print("Predicted selling price for Client {}'s home: ${:,.2f}".format(i+1, price)) 

Resale price: $255,000.00, $440,000.00, $385,000.00
Adj. inflation: $268,429.53, $463,172.52, $403,720.52
Predicted selling price for Client 1's home: $286,302.03
Predicted selling price for Client 2's home: $475,965.40
Predicted selling price for Client 3's home: $398,440.17
